In [11]:
import cv2
import numpy as np

# Load YOLO model
yolo_cfg = "yolov3.cfg"
yolo_weights = "yolov3.weights"
coco_names = "coco.names"

# Load class names
with open(coco_names, 'r') as f:
    classes = f.read().strip().split("\n")

# Load the YOLO network
net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# YOLO parameters
conf_threshold = 0.5
nms_threshold = 0.4

# Function to detect persons
def detect_persons(frame):
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    # Get YOLO layer names
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    # Forward pass
    outputs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    class_ids = []

    # Process YOLO output
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold and classes[class_id] == "person":
                # Scale box back to image size
                box = detection[0:4] * np.array([w, h, w, h])
                center_x, center_y, width, height = box.astype("int")

                x = int(center_x - width / 2)
                y = int(center_y - height / 2)

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    filtered_boxes = []
    if len(indices) > 0:
        # Check if indices are nested lists (older OpenCV versions)
        if isinstance(indices[0], (list, tuple)):
            for i in indices:
                filtered_boxes.append((boxes[i[0]], confidences[i[0]]))
        else:  # If indices are flat (newer OpenCV versions)
            for i in indices:
                filtered_boxes.append((boxes[i], confidences[i]))
    else:
        filtered_boxes = []
    
    return filtered_boxes

# Apply blur except closest person
def apply_blur_except_front(frame):
    detections = detect_persons(frame)
    if len(detections) == 0:
        return frame

    # Find the closest person (largest bounding box)
    largest_person = max(detections, key=lambda d: d[0][2] * d[0][3])[0]

    # Create a mask for blurring
    mask = np.zeros(frame.shape[:2], dtype="uint8")

    for (box, _) in detections:
        if box != largest_person:
            x, y, w, h = box
            cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)

    # Apply Gaussian blur to the masked area
    blurred_frame = cv2.GaussianBlur(frame, (25, 25), 0)
    frame = np.where(mask[..., None] == 255, blurred_frame, frame)

    # Draw a rectangle around the largest person
    for (box, _) in detections:
        x, y, w, h = box
        if box == largest_person:
            color = (0, 255, 0)  # Green for the closest person
        else:
            color = (0, 0, 255)  # Red for others
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 10)

    return frame

# Initialize webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply blur except closest person
    output_frame = apply_blur_except_front(frame)

    cv2.imshow("Blur Except Closest Person", output_frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# 휴대폰으로 촬영한 영상이 실시간으로 